In [23]:
import json
import ollama
import faiss
import numpy as np

# Đọc file JSON
with open('../data/Pacifist.products.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Khởi tạo danh sách để lưu embedding
embeddings_data = []

# Khởi tạo danh sách embedding cho Faiss
embedding_vectors = []
product_ids = []

# Duyệt qua từng sản phẩm và tạo embedding
for product in data:
    # Tạo nội dung để nhúng
    content = f"Tên: {product['productName']}\nGiá: {product['price']} VNĐ\nBộ sưu tập: {product['productCollection']}\nMã code: {product['productCode']}\nGiảm giá: {product['discount']}\nSize: {product['sizes']} "
    
    # Gọi Ollama để tạo embedding
    embedding_response = ollama.embeddings(
        model='nomic-embed-text',
        prompt=content
    )
    
    # Lấy vector embedding từ kết quả trả về
    embedding = embedding_response.get('embedding')
    if embedding is None:
        continue
    
    # Chuyển đổi embedding sang NumPy array
    embedding_vector = np.array(embedding, dtype='float32')
    
    # Lưu embedding cho Faiss
    embedding_vectors.append(embedding_vector)
    product_ids.append(product["_id"]["$oid"])
    
    # Lưu lại embedding kèm thông tin sản phẩm
    embeddings_data.append({
        "product_id": product["_id"]["$oid"],
        "product_name": product["productName"],
        "content": content,
        "embedding": embedding
    })

# Lưu dữ liệu embedding ra file JSON
with open('product_embeddings.json', 'w', encoding='utf-8') as f:
    json.dump(embeddings_data, f, ensure_ascii=False, indent=4)

print(f"Đã tạo embedding cho {len(embeddings_data)} sản phẩm.")

# Tạo index Faiss và lưu embedding vào Faiss
if len(embedding_vectors) > 0:
    # Xác định số chiều của embedding
    dimension = embedding_vectors[0].shape[0]
    
    # Tạo index bằng Faiss sử dụng L2 (Euclidean distance)
    index = faiss.IndexFlatL2(dimension)
    
    # Thêm các embedding vào index
    index.add(np.array(embedding_vectors))
    
    # Lưu index vào file
    faiss.write_index(index, 'faiss_product_index.index')
    
    print(f"Đã lưu {len(embedding_vectors)} embedding vào Faiss index.")


Đã tạo embedding cho 202 sản phẩm.
Đã lưu 202 embedding vào Faiss index.


In [32]:
with open('embeddings_data.txt', 'w', encoding='utf-8') as file:
    for item in embeddings_data:
        file.write(json.dumps(item, ensure_ascii=False) + '\n')

In [25]:
import json
import ollama
import faiss
import numpy as np

# Tải lại index từ file Faiss
index = faiss.read_index('../my-chatbot/faiss_product_index.index')

# Tạo truy vấn để lấy thông tin từ Faiss
query_content = "Cho tôi thông tin về các loại vòng tay"
query_embedding_response = ollama.embeddings(
    model='nomic-embed-text',
    prompt=query_content
)
query_embedding = query_embedding_response.get('embedding')

if query_embedding is not None:
    query_vector = np.array(query_embedding, dtype='float32')
    
    # Tìm kiếm trong Faiss
    k = 5  # Số lượng kết quả cần tìm
    distances, indices = index.search(np.array([query_vector]), k)
    
    # In ra kết quả tìm kiếm
    print("Kết quả tìm kiếm:")
    for i, idx in enumerate(indices[0]):
        if idx != -1:  # Kiểm tra xem index có hợp lệ hay không
            product_id = product_ids[idx]
            product_info = next((item for item in embeddings_data if item["product_id"] == product_id), None)
            if product_info:
                # print(f"Kết quả {i + 1}: {product_info['product_name']} (ID: {product_info['product_id']})")
                print(product_info['content'])
                print("################")
                # print(f"Khoảng cách: {distances[0][i]}")
else:
    print("Không thể tạo embedding cho truy vấn.")

Kết quả tìm kiếm:
Tên: Vòng Tay 1
Giá: 12300000 VNĐ
Bộ sưu tập: Giáng Sinh
Mã code: PVT000125
Giảm giá: 0
Size: [{'size': '4', 'quantity': 7, '_id': {'$oid': '6739d67ff01b50c5638b03e7'}}, {'size': '7', 'quantity': 2, '_id': {'$oid': '6739d67ff01b50c5638b03e8'}}] 
################
Tên: Nhẫn Test 1 
Giá: 1200000 VNĐ
Bộ sưu tập: Giáng Sinh 
Mã code: PN000200
Giảm giá: 12
Size: [{'size': '4', 'quantity': 1, '_id': {'$oid': '6739d886f01b50c5638b59fc'}}] 
################
Tên: Vòng tay Kim Cương and Turquoise Wire
Giá: 2450000 VNĐ
Bộ sưu tập: Pacifist T
Mã code: PVT00021
Giảm giá: 30
Size: [{'size': 6, 'quantity': 7}, {'size': 7, 'quantity': 4}, {'size': 8, 'quantity': 18}, {'size': 9, 'quantity': 12}, {'size': 10, 'quantity': 14}, {'size': 11, 'quantity': 17}, {'size': 12, 'quantity': 10}, {'size': 13, 'quantity': 16}, {'size': 14, 'quantity': 3}, {'size': 15, 'quantity': 4}] 
################
Tên: Vòng tay Kim Cương Wire
Giá: 2820000 VNĐ
Bộ sưu tập: Pacifist T
Mã code: PVT00020
Giảm giá: 2

In [20]:
def query_prompt(query):
    query_embedding_response = ollama.embeddings(
        model='nomic-embed-text',
        prompt=query
    )
    query_embedding = query_embedding_response.get('embedding')

    if query_embedding is not None:
        query_vector = np.array(query_embedding, dtype='float32')
        # Tìm kiếm trong Faiss
        k = 5  # Số lượng kết quả cần tìm
        distances, indices = index.search(np.array([query_vector]), k)
        
        # In ra kết quả tìm kiếm
        print("Kết quả tìm kiếm:")
        context = "\n"
        for i, idx in enumerate(indices[0]):
            if idx != -1:  # Kiểm tra xem index có hợp lệ hay không
                product_id = product_ids[idx]
                product_info = next((item for item in embeddings_data if item["product_id"] == product_id), None)
                if product_info:
                    context += product_info['content'] + "\n################"
    else:
        print("Không thể tạo embedding cho truy vấn.")
    
    prompt = f"""
You are an AI assistant specialized in providing detailed and contextually relevant answers. Use the following context and user query to generate your response in Vietnamese:

Context: {context}

Query: {query}

Based on the context and query, provide a concise and accurate answer in Vietnamese.
    """


    return prompt

In [21]:
print(query_prompt("Cho tôi thông tin về các loại vòng tay bằng vàng"))

Kết quả tìm kiếm:

You are an AI assistant specialized in providing detailed and contextually relevant answers. Use the following context and user query to generate your response in Vietnamese:

Context: 
Tên: Vòng Tay 1
Giá: 12300000 VNĐ
Bộ sưu tập: Giáng Sinh
Mã code: PVT000125
Giảm giá: 0
Size: [{'size': '4', 'quantity': 7, '_id': {'$oid': '6739d67ff01b50c5638b03e7'}}, {'size': '7', 'quantity': 2, '_id': {'$oid': '6739d67ff01b50c5638b03e8'}}] 
################Tên: Nhẫn Test 1 
Giá: 1200000 VNĐ
Bộ sưu tập: Giáng Sinh 
Mã code: PN000200
Giảm giá: 12
Size: [{'size': '4', 'quantity': 1, '_id': {'$oid': '6739d886f01b50c5638b59fc'}}] 
################Tên: Vòng tay Full Heart Toggle
Giá: 11950000 VNĐ
Bộ sưu tập: Return to Pacifist
Mã code: PVT00053
Giảm giá: 5
Size: [{'size': 6, 'quantity': 17}, {'size': 7, 'quantity': 12}, {'size': 8, 'quantity': 5}, {'size': 9, 'quantity': 10}, {'size': 10, 'quantity': 11}, {'size': 11, 'quantity': 3}, {'size': 12, 'quantity': 2}, {'size': 13, 'quantity'

In [4]:
import json
import requests

# NOTE: ollama must be running for this to work, start the ollama app or run `ollama serve`
model = 'llama3.2:3b'

def generate(prompt):
    r = requests.post('http://localhost:11434/api/generate',
                      json={
                          'model': model,
                          'prompt': prompt
                      },
                      stream=True)
    r.raise_for_status()

    response_text = ""
    for line in r.iter_lines():
        body = json.loads(line)
        response_part = body.get('response', '')
        # the response streams one token at a time, append that as we receive it
        response_text += response_part

        if 'error' in body:
            raise Exception(body['error'])

        if body.get('done', False):
            return response_text